Convert to MusicXML and MIDI

In [ ]:
import music21 as m21
import pandas as pd
import os

def convert_song_to_midi(song : m21.stream.Score):
    sXML = m21.stream.Stream()
    sMIDI = m21.stream.Stream()
    for part_id, part in enumerate(song.parts):
        print(part_id)
        sXML.append(part)
        p = m21.stream.Part()
        p.insert(m21.instrument.Bagpipes())
        print(part.partName)
        for event in part.flat.notesAndRests:
            if isinstance(event, m21.note.Note):
                print(event.pitch.midi, end=' ')
                print(event.duration.quarterLength, end='|')
                p.append(m21.note.Note(int(event.pitch.midi), quarterLength=event.duration.quarterLength))
            else:
                print("r", end='')
                print(event.duration.quarterLength)
                p.append(m21.note.Rest(quarterLength=event.duration.quarterLength))
        sMIDI.append(p)
        
    sXML.write('musicxml', f'{song.metadata.bestTitle}')
    sMIDI.write('midi', f'{song.metadata.bestTitle}.mid')
    
paths = m21.corpus.getComposer('bach')
songs = []
# for path in paths[0]:
song = m21.corpus.parse(paths[5])
songs.append(song)
convert_song_to_midi(songs[0])

In [15]:
import music21 as m21
import pandas as pd
import os

part_indexes = {
    0: 'soprano',
    1: 'alto',
    2: 'tenor',
    3: 'bass'    
}

def make_measure_chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    return [lst[i:i+n] for i in range(len(lst)-n+1)]

def to_dataset(songs):
    with open('dataset.csv', 'w') as fd:
        fd.write('sixteenth,soprano,alto,tenor,bass,file\n')        
        for song in songs:
            print(f'Processing {song.metadata.bestTitle}...')
            parts = {}
            print(f"Found {len(song.parts)} parts")
            if len(song.parts) < 4 or len(song.parts) > 4:
                print("Skip")
                continue
            # iterate over the 4 parts
            for part_id, part in enumerate(song.parts):
                # Get all the measures in this part
                measures = [mm for mm in part.getElementsByClass('Measure') if mm.number != 0]
                # Group them into groups of 4
                measure_chunks = make_measure_chunks(measures, 4)
                part_chunks = []
                for chunk in measure_chunks:
                    # For every chunk of 4 measures
                    chunk_encoding = {(offset/4.0): '--' for offset in range(0, 128)} #TODO Back to 64
                    for measure_id, measure in enumerate(chunk):
                        # Iterate over each measure
                        for ev in measure:
                            # And every note within the measure
                            offs = 4.0 * measure_id + ev.offset
                            if isinstance(ev, m21.chord.Chord):                        
                                if offs in chunk_encoding:
                                    chunk_encoding[offs] = ev[0].nameWithOctave
                            elif isinstance(ev, m21.note.Note):
                                if offs in chunk_encoding:
                                    chunk_encoding[offs] = ev.nameWithOctave                        
                            elif isinstance(ev, m21.note.Rest):                        
                                if offs in chunk_encoding:
                                    chunk_encoding[offs] = 'Rest'
                                    
                    part_chunks.append(list(chunk_encoding.values()))
                parts[part_indexes[part_id]] = part_chunks
                
            for chunk_id in range(len(parts['soprano'])):
                dfdict = {}
                for part, chunks in parts.items():
                    dfdict[part] = chunks[chunk_id]
                chunk_name = f'{song.metadata.bestTitle}_chunk_{chunk_id}'
                dfdict['file'] = chunk_name            
                df = pd.DataFrame(dfdict)
                df.to_csv(fd, header=False)
    print('The dataset.csv file has been written!')
    
paths = m21.corpus.getComposer('bach')
songs = []
for path in paths:
    song = m21.corpus.parse(path)
    songs.append(song)
to_dataset(songs)

Processing bwv1.6.mxl...
Found 5 parts
Skip
Processing bwv10.7.mxl...
Found 4 parts
Processing bwv101.7.mxl...
Found 4 parts
Processing bwv102.7.mxl...
Found 4 parts
Processing bwv103.6.mxl...
Found 4 parts
Processing bwv104.6.mxl...
Found 4 parts
Processing bwv108.6.mxl...
Found 4 parts
Processing bwv11.6.mxl...
Found 4 parts
Processing bwv110.7.mxl...
Found 4 parts
Processing bwv111.6.mxl...
Found 4 parts
Processing bwv112.5-sc.mxl...
Found 7 parts
Skip
Processing bwv112.5.mxl...
Found 4 parts
Processing bwv113.8.mxl...
Found 4 parts
Processing bwv114.7.mxl...
Found 4 parts
Processing bwv115.6.mxl...
Found 4 parts
Processing bwv116.6.mxl...
Found 4 parts
Processing bwv117.4.mxl...
Found 4 parts
Processing bwv119.9.mxl...
Found 4 parts
Processing bwv12.7.mxl...
Found 5 parts
Skip
Processing bwv120.6.mxl...
Found 4 parts
Processing bwv120.8-a.mxl...
Found 8 parts
Skip
Processing bwv121.6.mxl...
Found 4 parts
Processing bwv122.6.mxl...
Found 4 parts
Processing bwv123.6.mxl...
Found 4 pa